In [1]:
import numpy as np
import pandas as pd
import scipy.stats as st
from causallearn.search.ScoreBased.GES import ges
import itertools
from sklearn.preprocessing import StandardScaler
from dagma.nonlinear import DagmaMLP, DagmaNonlinear
import torch


/Users/shah38/Desktop/lucid_cd/lucid_env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv("data/cd_matrix_dA.csv")
X = df.values
print(df.head())

   MYORG  WDR44  NSA2  MRPS26  PTGS2  IFT52  MAF  LOC105377627  DAPK3  MKNK2  \
0   1140   1618  3404    1157   2288   1550  128           190   5910   3456   
1   1402   2213  5139    1574   1413   2219  118            73   5513   8203   
2   1123   1785  4566    1190   1381   2008  138           252   5579   3954   
3   1126   1386  3888    1012    539   1954   70           172   5282   3925   
4    836   1487  2848     828   2276   1246   68           188   4821   2950   

   ...  GNG12  ELOVL1  FAM171A1  PDE1C  ZNF22   RPSA  CD40  NOTCH2  IGFBP3  \
0  ...  31498    7852      8311   1410   2625  41014   424    8598     322   
1  ...  27016    7743      7952    601   5038  49442  2387    8050     769   
2  ...  38299    8736      9994   1516   2737  54805   334   11262     418   
3  ...  29153    6684      8090   1225   2429  46397   474    7337     280   
4  ...  25858    6523      6913   1251   2108  34177   330    7163     242   

   radiation  
0        0.0  
1        0.0  
2    

In [3]:
X_norm = StandardScaler().fit_transform(X)
print(X_norm)

[[ 0.27441481 -0.29823198 -1.37249679 ... -0.30968512 -0.85772296
  -0.80903983]
 [ 1.80663918  2.27347858  1.15416539 ... -0.56367968  2.19446817
  -0.80903983]
 [ 0.17499568  0.42357586  0.31971154 ...  0.92506217 -0.20221882
  -0.80903983]
 ...
 [-1.33383594 -0.31984299  0.01971592 ...  0.0286653  -0.5709399
   0.94387981]
 [-0.4039746  -0.40196484  1.32455126 ...  0.2947107   0.83566272
   1.82033963]
 [-0.70808019 -0.9897844   0.30369236 ... -0.23274516  0.65813035
   1.82033963]]


In [4]:
device = torch.device("mps" if torch.mps.is_available() else "cpu")
print(device)
d = X_norm.shape[1]
X_norm_tensor = torch.from_numpy(X).type(torch.float).to(device)
eq_model = DagmaMLP(dims=[d, 10, 1], bias=True, dtype=torch.float) # create the model for the structural equations, in this case MLPs
eq_model = eq_model.to(device)
print(eq_model.I.device)
model = DagmaNonlinear(eq_model, dtype=torch.float, device=device) # create the model for DAG learning
W_est = model.fit(X_norm, lambda1=0.02, lambda2=0.005) # fit the model with L1 reg. (coeff. 0.02) and L2 reg. (coeff. 0.005)

mps
mps:0


  0%|          | 0/230000.0 [00:00<?, ?it/s]


NotImplementedError: The operator 'aten::_linalg_slogdet.sign' is not currently implemented for the MPS device. If you want this op to be considered for addition please comment on https://github.com/pytorch/pytorch/issues/141287 and mention use-case, that resulted in missing op as well as commit hash 2236df1770800ffea5697b11b0bb0d910b2e59e1. As a temporary fix, you can set the environment variable `PYTORCH_ENABLE_MPS_FALLBACK=1` to use the CPU as a fallback for this op. WARNING: this will be slower than running natively on MPS.